In [5]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator,os
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from dotenv import load_dotenv

In [15]:
load_dotenv(override=True)
api_key=os.getenv('OPENAI_API_KEY')
tavily_api_key = os.getenv('TAVILY_API_KEY')

In [16]:
tool = TavilySearchResults(max_results = 4)

In [17]:
print(type(tool))

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>


In [20]:
class AgentState(TypedDict):
    messages : Annotated[list[AnyMessage] , operator.add]

In [35]:
class Agent:
    def __init__(self,model,tools,system):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node('llm',self.call_llm)
        graph.add_node('action',self.take_action)
        graph.add_conditional_edges('llm',self.exist_action,{True:'action',False:END})
        graph.add_edge('action','llm')
        graph.set_entry_point('llm')
        self.graph = graph.compile()
        self.tools = {t.name : t for t in tools}
        self.model = model.bind_tools(tools)

    def call_llm(self, state : AgentState):
        messages = state['messages']
        if self.system :
            messages = [SystemMessage(content = self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages' : [message]}

    def take_action(self , state : AgentState):
        tool_calls = state ['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling Action: {t}")
            if not t['name'] in self.tools:
                print("\n .....bad tool name .....")
                result = "bad tool name , retry"
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to model !!")
        return {'messages' : results}

    def exist_action(self,state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0
    
        
    

In [36]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

In [37]:
model = ChatOpenAI(model = "gpt-3.5-turbo")
bot = Agent(model,[tool],system = prompt)

In [38]:
messages = [HumanMessage(content="What is the weather in sf?")]
result = bot.graph.invoke({"messages": messages})

Calling Action: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_tcimQdoq1THBWWHMQGucgBTr', 'type': 'tool_call'}
Back to model !!


In [ ]:
print(result['messages'][-1].content)

In [41]:
query = "Who won the super bowl in 2024? In what state is the winning team headquarters located? \
What is the GDP of that state? Answer each question." 
messages = [HumanMessage(content=query)]

model = ChatOpenAI(model="gpt-4o")  
abot = Agent(model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

Calling Action: {'name': 'tavily_search_results_json', 'args': {'query': '2024 Super Bowl winner'}, 'id': 'call_ggvc4r2vn3PyEsxIqQUMgLpx', 'type': 'tool_call'}
Back to model !!
Calling Action: {'name': 'tavily_search_results_json', 'args': {'query': 'Kansas City Chiefs headquarters location'}, 'id': 'call_cgdbTxnCxeLc9Uq38lJdgvmt', 'type': 'tool_call'}
Calling Action: {'name': 'tavily_search_results_json', 'args': {'query': 'GDP of Missouri 2024'}, 'id': 'call_VZJtH8mt0P6iTAijAIWqTWhH', 'type': 'tool_call'}
Back to model !!


In [42]:
print(result['messages'][-1].content)

1. **Who won the Super Bowl in 2024?**

   The Kansas City Chiefs won the Super Bowl in 2024, defeating the San Francisco 49ers 25-22 in overtime.

2. **In what state is the winning team's headquarters located?**

   The Kansas City Chiefs' headquarters is located in Missouri.

3. **What is the GDP of that state?**

   The GDP of Missouri as of 2024 is approximately $34.3 trillion.
